In [44]:
# INTERFACE GRAPHIQUE
from tkinter import*
from tkinter import filedialog
from tkinter import messagebox

root = Tk()
root.config(bg='#9370DB')
message = Label(root,text="Veuillez rentrer les paramètres de l'option."
                ,font=("Century Gothic", 15, 'bold'),bg='#9370DB',fg='white')
                
message.pack(ipadx=10,ipady=10)

# window size
root.title("Pricer")
window_width=900
window_height=500

# screen size
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# find the center point
center_x = int(screen_width/2 - window_width / 2)
center_y = int(screen_height/2 - window_height / 2)

# set the position of the window to the center of the screen
root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')


# CREATION FRAME_INPUT POUR LES DONNEES A RENTRER 

frame_input = Frame(root)
frame_choose = Frame(root)

label_fichier = Label(frame_input,text="Fichier csv :", bg="#9370DB",fg='white',
                   font=("Century Gothic",10))
label_strike = Label(frame_input,text="Strike :",bg="#9370DB",fg='white',
                   font=("Century Gothic",10))
label_spot = Label(frame_input,text="Spot :",bg="#9370DB",fg='white',
                   font=("Century Gothic",10))
label_rf = Label(frame_input,text="Taux sans risque :",bg="#9370DB",fg='white',
                   font=("Century Gothic",10))
label_div = Label(frame_input,text="Dividende :",bg="#9370DB",fg='white',
                   font=("Century Gothic",10))
label_maturite = Label(frame_input,text="Maturité :",bg="#9370DB",fg='white',
                   font=("Century Gothic",10))

# Récupération du fichier csv et calcul volatilite
def openFile():
    filepath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    file = open(filepath,'r')
    #print(file.read())
    file.close()

def import_closing_prices():
    close_data= pd.read_csv(file,usecols=['AdjClose'])
    print(close_data)
    returns_histo = close_data.pct_change()
    volatilite = np.std(returns_histo)
    return volatilite

button_import = Button(frame_input,text="Parcourir",width=20,command=openFile,bg="white",fg='#9370DB',
                   font=("Century Gothic",10))
                   
# création des champs d'entrée pour frame_input

entry_strike = Entry(frame_input,width=30)
entry_spot = Entry(frame_input,width=30)
entry_rf = Entry(frame_input,width=30)
entry_div = Entry(frame_input,width=30)
entry_maturite = Entry(frame_input,width=30)

# Placement des éléments dans les frames

label_fichier.grid(row=0,column=0,padx=30,pady=20)
label_strike.grid(row=1,column=0,padx=25,pady=15)
label_spot.grid(row=2,column=0,padx=25,pady=15)
label_rf.grid(row=3,column=0,padx=25,pady=15) 
label_div.grid(row=4,column=0,padx=25,pady=15)
label_maturite.grid(row=5,column=0,padx=30,pady=20) 

button_import.grid(row=0,column=1,columnspan=2)
entry_strike.grid(row=1,column=1)
entry_spot.grid(row=2,column=1)
entry_rf.grid(row=3,column=1)
entry_div.grid(row=4,column=1)
entry_maturite.grid(row=5,column=1)

 # CREATION FRAME_CHOOSE POUR LES RADIOBOX
 
label_type = Label(frame_choose,text="Choisir le type d'option",
                   bg="#9370DB",fg='white',
                   font=("Century Gothic",12,'bold')).pack(side=TOP,ipadx=10,ipady=10,fill='x')

global option
option = IntVar()

# fonctions pour récupérer les choix radiobox
def selected_option():
    #print(option.get())
    option_type = option.get()
    return option_type

bouton1=Radiobutton(frame_choose, text="Option Américaine",variable=option,value=1,
                    bg="#9370DB",fg='white', selectcolor='#9370DB',
                   font=("Century Gothic",10),command=selected_option)
bouton2 = Radiobutton(frame_choose,text="Option Européenne",variable=option,value=2,
                      bg="#9370DB",fg='white',selectcolor='#9370DB', 
                   font=("Century Gothic",10),command=selected_option)

bouton1.pack(side=TOP,ipadx=10,ipady=5)
bouton2.pack(side=TOP,ipadx=10,ipady=5)

# Résultats
call_price = Label(frame_choose,text="Prix du call",
                      bg="#9370DB",fg='white', font=("Century Gothic",10))
                      
put_price = Label(frame_choose,text="Prix du put",
                      bg="#9370DB",fg='white', font=("Century Gothic",10))
                      
call_price.pack(side=TOP,ipadx=10,ipady=5)
value_call = Entry(frame_choose,bg="white",fg='#9370DB').pack(side=TOP)                  

put_price.pack(side=TOP,ipadx=10,ipady=5)  
value_put = Entry(frame_choose,bg="white",fg='#9370DB').pack(side=TOP)
                      

                  
 # PLACEMENT DANS LA FENETRE PRINCIPALE
    
# Placement des frames dans la fenêtre et mise en forme

frame_input.pack(side=LEFT,padx=80,pady=10,fill='both')
frame_input.config(bg='#9370DB')

frame_choose.pack(side=TOP,padx=35,pady=10,fill='both')
frame_choose.config(bg="#9370DB")   



#################################################################################################"

# PRICER BLACK & SCHOLES
import pandas as pd
import numpy as np
import math as m

#S: prix du spot
#K: prix du strike 
#T: maturité
#r: taux d'intérêt sans risque
#D: taux de dividende continu versé par le sous-jacent
#sigma: volatilité du sous-jacent
    
#frame_choose.pack
S = DoubleVar(entry_spot.get())
K = DoubleVar(entry_strike.get())
T = DoubleVar(entry_maturite.get())
r = DoubleVar(entry_rf.get())
D = DoubleVar(entry_div.get())
sigma = import_closing_prices
#sigma = volatilite
    
# pricing options 
def black_scholes_call(S, K, T, r, D, sigma):
    
    d1 = (np.log(S / K) + (r - D + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - D - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * np.exp(-D * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call


# pricing put
def black_scholes_put(S, K, T, r, D, sigma):
    
    d1 = (np.log(S / K) + (r - D + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - D - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    put = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * np.exp(-D * T) * si.norm.cdf(-d1, 0.0, 1.0))
    
    return put


def Get_results():
    choix_option = option.get()
    
    if choix_option == 2:
        value_call.insert(0,black_scholes_call(S, K, T, r, D, sigma))
        value_put.insert(0,black_scholes_put(S, K, T, r, D, sigma))
    elif choix_option == 1:
        messagebox.showerror("Erreur","Le pricer Black&Scholes ne permet pas de calculer le prix des options Amércaines.")

# bouton final à cliquer 

bouton_price = Button(root,text="Lancer le pricer",height=10,width=30, bg="white",fg='#9370DB',
                   font=("Century Gothic",10),command=Get_results)

bouton_price.pack(side=BOTTOM,padx=50,pady=50)


root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Avel ANDRIAM\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\AVELAN~1\AppData\Local\Temp/ipykernel_25028/958865099.py", line 190, in Get_results
    value_call.insert(0,black_scholes_call(S, K, T, r, D, sigma))
AttributeError: 'NoneType' object has no attribute 'insert'
